# install dataset

In [1]:
!pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
!chmod 600 /root/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
!kaggle datasets download meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
 99% 605M/612M [00:10<00:00, 91.0MB/s]
100% 612M/612M [00:10<00:00, 60.4MB/s]


In [6]:
! unzip gtsrb-german-traffic-sign.zip -d "/content/dataset"

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: /content/dataset/train/5/00005_00053_00010.png  
  inflating: /content/dataset/train/5/00005_00053_00011.png  
  inflating: /content/dataset/train/5/00005_00053_00012.png  
  inflating: /content/dataset/train/5/00005_00053_00013.png  
  inflating: /content/dataset/train/5/00005_00053_00014.png  
  inflating: /content/dataset/train/5/00005_00053_00015.png  
  inflating: /content/dataset/train/5/00005_00053_00016.png  
  inflating: /content/dataset/train/5/00005_00053_00017.png  
  inflating: /content/dataset/train/5/00005_00053_00018.png  
  inflating: /content/dataset/train/5/00005_00053_00019.png  
  inflating: /content/dataset/train/5/00005_00053_00020.png  
  inflating: /content/dataset/train/5/00005_00053_00021.png  
  inflating: /content/dataset/train/5/00005_00053_00022.png  
  inflating: /content/dataset/train/5/00005_00053_00023.png  
  inflating: /content/dataset/train/5/00005_00053_00024.png  
  inflating: 

# CNN

# RF

In [7]:
import os
import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import pandas as pd
from skimage import color
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [8]:
# Đường dẫn đến dataset
dataset_dir = "/content/dataset"
train_csv_path = os.path.join(dataset_dir, '/content/dataset/Train.csv')
test_csv_path = os.path.join(dataset_dir, '/content/dataset/Test.csv')

In [9]:
# Hàm để đọc ảnh và xử lý nhãn từ file CSV
def load_data_from_csv(data_path, dataset_dir):
    images = []
    labels = []
    # Đọc file CSV chứa nhãn
    data = pd.read_csv(data_path)
    for index, row in data.iterrows():
        img_path = os.path.join(dataset_dir, row['Path'])
        img = cv2.imread(img_path)
        if img is not None:
            # Resize, chuyển sang grayscale và trích xuất đặc trưng HOG
            img_gray = color.rgb2gray(cv2.resize(img, (64, 64)))  # Tăng kích thước ảnh
            features = hog(img_gray, block_norm='L2-Hys', pixels_per_cell=(8, 8), cells_per_block=(2, 2))
            images.append(features)  # Append features extracted from the current image
            labels.append(row['ClassId'])  # Nhãn được lấy từ cột 'ClassId' trong CSV

    return np.array(images), np.array(labels) # Return the accumulated images list

In [10]:
# Tải dữ liệu từ file CSV
X_train, y_train = load_data_from_csv(train_csv_path, dataset_dir)
X_test, y_test = load_data_from_csv(test_csv_path, dataset_dir)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split



# Giảm số lượng tổ hợp tham số
param_grid = {
    'n_estimators': [10, 50, 100],   # Giảm số lượng cây ban đầu
    'max_depth': [10, 20, None],     # Giảm số lựa chọn độ sâu
}

# Khởi tạo RandomizedSearchCV để tìm kiếm ngẫu nhiên với số lần ít hơn
clf = RandomForestClassifier(random_state=42)
grid_search = RandomizedSearchCV(estimator=clf, param_distributions=param_grid, n_iter=5, cv=2, n_jobs=-1, verbose=2, random_state=42)

# Huấn luyện mô hình với tập mẫu nhỏ
grid_search.fit(X_train, y_train)

# Lấy mô hình tốt nhất sau khi tìm kiếm
best_clf = grid_search.best_estimator_

# Dự đoán trên tập kiểm tra (với mô hình được huấn luyện lại từ toàn bộ dữ liệu nếu cần)
y_pred = best_clf.predict(X_test)


Fitting 2 folds for each of 5 candidates, totalling 10 fits


In [14]:
# Đánh giá kết quả
print("Classification report for RF:\n", classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

Classification report for RF:
               precision    recall  f1-score   support

           0       1.00      0.60      0.75        60
           1       0.91      0.81      0.86       720
           2       0.79      0.97      0.87       750
           3       0.94      0.79      0.86       450
           4       0.91      0.96      0.93       660
           5       0.80      0.83      0.81       630
           6       0.82      0.74      0.78       150
           7       0.90      0.92      0.91       450
           8       0.86      0.86      0.86       450
           9       0.90      0.99      0.94       480
          10       0.96      0.99      0.98       660
          11       0.78      0.94      0.85       420
          12       0.99      1.00      0.99       690
          13       1.00      1.00      1.00       720
          14       0.98      0.94      0.96       270
          15       0.92      0.96      0.94       210
          16       1.00      0.95      0.97       

In [15]:
# In ra các tham số tốt nhất
print("Best Parameters found by RandomizedSearchCV:")
print(grid_search.best_params_)

# In ra mô hình tốt nhất
print("\nBest Estimator (model) found by RandomizedSearchCV:")
print(grid_search.best_estimator_)

# In ra điểm đánh giá tốt nhất
print("\nBest Score achieved during RandomizedSearchCV:")
print(grid_search.best_score_)


Best Parameters found by RandomizedSearchCV:
{'n_estimators': 100, 'max_depth': None}

Best Estimator (model) found by RandomizedSearchCV:
RandomForestClassifier(random_state=42)

Best Score achieved during RandomizedSearchCV:
0.9388900224443991


In [16]:
# In ra số lượng cây (n_estimators)
print("Number of trees (n_estimators):", best_clf.n_estimators)

# In ra độ sâu của cây (max_depth)
print("Max depth of trees (max_depth):", best_clf.max_depth)


Number of trees (n_estimators): 100
Max depth of trees (max_depth): None
